# updates

1. 从polygon采样阶段将time设置为yyyy-mm-dd,确保ee.filter.date()能够采集合理日期合适，Date格式
2. 增加sentinel-1 filter中and 筛选条件，由于ee,filter只能筛选指定字符，没有与门逻辑
3. 可视化参数 v isParamsNorm 中的 min 和 max 参数是固定的，但归一化的结果范围可能不是 [0, 2]，应根据统计值动态调整：
4. 将采样结果导出为csv存储，再进行随机森林分类


**Notice**


time format in Arcgis pro is Date, unreadable, for read should use ee.format('yyyy-mm-dd')

featureCollection.filter('time'equal date),这里的date format还是unreadable long, 需要保持一致，

time 的使用，筛选需要按照函数要求变换

2025/1/22


**updates 2025/2/15**

filter imageCollection by specific name,

extract the list name beforehead

using list loop to save calculation

[filter reference link](https://github.com/gee-community/geemap/discussions/584)

**updates 2025/2/24**

sample region function turns out the sampling extent is too large and exceed the calculation on gee

to solve: 1. random sample (not work, the size of polygons varies and the random seed is constant) ×

  2. generate the point in polygon first √

**updates 2025/2/25**

add HH_norm, HV_norm, six bands totally

**skip all test**

In [1]:
import ee
import geemap
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd


## Initialize Earth Engine

In [14]:
import os

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
# Map = geemap.Map(height=800,width=800)


In [15]:
geemap.set_proxy(port=7890)
Map=geemap.Map()#设置地图大小

In [16]:
Greenland_North = ee.FeatureCollection('projects/ee-my-yujiesun/assets/ee-my-SeaIce/SeaIce_Boundary/Greenland_North')
Sample_2018910 = ee.FeatureCollection('projects/ee-my-yujiesun/assets/ee-my-SeaIce/SeaIce_Samples/Sample_20180910_lead_addition_Points')
Greenland_NorthEast = ee.FeatureCollection('projects/ee-my-yujiesun/assets/ee-my-SeaIce/SeaIce_Boundary/Greenland_NorthEast')
Greenland_NorthWest = ee.FeatureCollection('projects/ee-my-yujiesun/assets/ee-my-SeaIce/SeaIce_Boundary/Greenland_NorthWest')
# sample_polygon = geemap.ee_to_geopandas(Sample_2018910)
# Greenland_North = geemap.ee_to_geopandas(Greenland_North)
sample_polygon = Sample_2018910
region = Greenland_NorthEast.merge(Greenland_NorthWest)
print(Sample_2018910.getInfo())
Map.addLayer(region, {}, 'region')
Map.addLayer(Sample_2018910, {}, 'Sample_2018910')
Map 

{'type': 'FeatureCollection', 'columns': {'CC': 'Float', 'class': 'Long', 'class_name': 'String', 'form': 'Float', 'source': 'String', 'stage': 'Float', 'system:index': 'String', 'time': 'Long', 'time_t': 'String'}, 'version': 1740544835522897, 'id': 'projects/ee-my-yujiesun/assets/ee-my-SeaIce/SeaIce_Samples/Sample_20180910_lead_addition_Points', 'properties': {'system:asset_size': 76359}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-21.367245282731457, 75.44224650214743]}, 'id': '00000000000000000000', 'properties': {'CC': None, 'class': 1, 'class_name': 'lead', 'form': None, 'source': '20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon', 'stage': None, 'time': 1535958000000, 'time_t': '2018-09-03'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-21.37813440750397, 75.47589041959567]}, 'id': '00000000000000000001', 'properties': {'CC': None, 'class': 1, 'class_name': 'lead', 'form': None, 'source': '20180903T082725_S1B

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## test region filter

In [13]:
start_date = '2018-10-28'
end_date = '2018-10-30'

sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(region)
             .filter(ee.Filter.eq('instrumentMode', 'EW'))  # Only EW mode
             .filterDate(start_date, end_date)
             .filterMetadata("system:index","contains", "20181029")
             .filter(ee.Filter.Or(
                 ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'),
                 ee.Filter.listContains('transmitterReceiverPolarisation', 'HV')
             )))

Map.addLayer(sentinel1, {}, 'sentinel1')
Map

Map(bottom=680.0, center=[74.21198251594369, -26.894531250000004], controls=(WidgetControl(options=['position'…

## group the polygon by time

shp property on gee, date format is convert to long, ee.date() can solve the unreadable problem

In [6]:
time = sample_polygon.first().get('time').getInfo()
time = ee.Date(time).format('YYYY-MM-dd').getInfo()
time


'2018-09-03'

## Sentienl-1 processing

### testing

In [7]:
unique_dates = sample_polygon.aggregate_array('time').distinct().getInfo()  # 'time' 为 Unix 时间戳
formatted_dates = [ee.Date(date).format('YYYY-MM-dd').getInfo() for date in unique_dates]
print(f"Unique dates: {unique_dates}")

Unique dates: [1535958000000, 1536562800000, 1536649200000, 1536908400000, 1536994800000, 1537772400000, 1538204400000, 1538636400000, 1539068400000, 1540191600000, 1540796400000, 1658127600000]


In [8]:
print(formatted_dates)

['2018-09-03', '2018-09-10', '2018-09-11', '2018-09-14', '2018-09-15', '2018-09-24', '2018-09-29', '2018-10-04', '2018-10-09', '2018-10-22', '2018-10-29', '2022-07-18']


In [8]:
S1_list = sample_polygon.aggregate_array('source').distinct().getInfo()
S1_list = [str(i)[:15] for i in S1_list] 
print(f"S1_list: {S1_list}")

S1_list: ['20180911T184432', '20180914T074628', '20180915T082725', '20180924T080254', '20180929T081103', '20181004T180321', '20181009T181232', '20181022T180529', '20181023T121709', '20180903T082725', '20180910T081814', '20180910T081914', '20181023T121814', '20220718']


In [11]:
start_date = '2018-09-01'
end_date = '2018-09-05'
name = '20180903T082725' in S1_list
# Load the Sentinel-1 ImageCollection for EW mode (Extra Wide) near the Arctic
sentinel1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(region)
             .filter(ee.Filter.eq('instrumentMode', 'EW'))  # Only EW mode
             .filterDate(start_date, end_date)
             .filterMetadata("system:index","contains", "20180903T082725")
             .filter(ee.Filter.Or(
                 ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'),
                 ee.Filter.listContains('transmitterReceiverPolarisation', 'HV')
             )))
             
print(sentinel1.size().getInfo())
print(sentinel1.aggregate_array('system:index').getInfo())
print(name)

4
['S1B_EW_GRDM_1SDH_20180903T082625_20180903T082725_012548_017260_966D', 'S1B_EW_GRDM_1SDH_20180903T082625_20180903T082725_012548_017260_9BC2', 'S1B_EW_GRDM_1SDH_20180903T082725_20180903T082825_012548_017260_7FCB', 'S1B_EW_GRDM_1SDH_20180903T082725_20180903T082825_012548_017260_B22E']
True


In [12]:
s1 = sentinel1.median()
diff_HH_HV = s1.select('HH').subtract(s1.select('HV')).rename('diff_HH_HV')
div_HH_HV = s1.select('HH').divide(s1.select('HV')).rename('div_HH_HV')
diff_HH_HV

### function

band selection

In [17]:
def normalize_band(image, band_name):
    """
    对指定波段进行归一化处理。
    """
    band = image.select(band_name)
    stats = band.reduceRegion(
        reducer=ee.Reducer.minMax(),  # 同时计算最小值和最大值
        geometry=region,
        scale=1000,
        maxPixels=1e8
    )
    min_val = ee.Number(stats.get(band_name + '_min'))
    max_val = ee.Number(stats.get(band_name + '_max'))

    normalized = band.subtract(min_val).divide(max_val.subtract(min_val)).rename(band_name + '_norm')
    return normalized

# safe_select make sue only when bands are available, then select them
def safe_select(image, bands):
    available_bands = image.bandNames()
    filtered_bands = ee.List(bands).filter(lambda band: available_bands.contains(band))
    if filtered_bands.size().getInfo() == 0:
        raise ValueError(f"None of the specified bands {bands} are available in the image.")
    return image.select(filtered_bands)


# Sentinel-1 图像处理函数
def process_s1_image(image):
    diff_HH_HV = image.select('HH').subtract(image.select('HV')).rename('diff_HH_HV')
    div_HH_HV = image.select('HH').divide(image.select('HV')).rename('div_HH_HV')
    image_n = image.addBands(diff_HH_HV).addBands(div_HH_HV);
   # image_n = image.addBands([diff_HH_HV, div_HH_HV])
    
    # # 归一化每个波段
    # bands_to_normalize = ['HH', 'HV', 'diff_HH_HV', 'div_HH_HV']
    # normalized_bands = [normalize_band(image_n, band) for band in bands_to_normalize]
    # return image_n.addBands(normalized_bands)
    
     # 归一化每个波段
    HH_norm = normalize_band(image, 'HH')
    HV_norm = normalize_band(image, 'HV')
    diff_HH_HV_norm = normalize_band(image_n , 'diff_HH_HV')
    div_HH_HV_norm = normalize_band(image_n , 'div_HH_HV')    
    
    return image_n.addBands([HH_norm, HV_norm, diff_HH_HV_norm, div_HH_HV_norm])

In [18]:
def get_vis_params(image, bands):
    """
    动态生成归一化波段的可视化参数。
    """
    vis_stats = image.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=region,
        scale=1000,
        maxPixels=1e8
    )
    min_val = min([vis_stats.get(band + '_min').getInfo() for band in bands])
    max_val = max([vis_stats.get(band + '_max').getInfo() for band in bands])
    return {
        'bands': bands,
        'min': min_val,
        'max': max_val,
        'gamma': 1
    }

def visualize_issue(date_ms, sample_polygon, sentinel1_collection):
    """
    可视化特定日期的 Sentinel-1 图像和多边形分布，用于排查问题。
    
    Args:
    - date_ms: 日期的 Unix 时间戳（毫秒）。
    - sample_polygon: ee.FeatureCollection，多边形集合。
    - sentinel1_collection: ee.ImageCollection，Sentinel-1 图像集合。
    """
    # 定义日期范围
    date = ee.Date(date_ms)
    date_start = date.advance(-2, 'day')
    date_end = date.advance(2, 'day')
    print(f"Visualizing for date: {date.format('YYYY-MM-dd').getInfo()}")

    # 筛选特定日期范围内的影像
    images_filtered = sentinel1_collection.filterDate(date_start, date_end)
    print(f"Number of images for date: {images_filtered.size().getInfo()}")

    # 筛选覆盖区域的多边形
    polygons_filtered = sample_polygon.filter(ee.Filter.eq('time', date_ms)) 
    print(f"Number of polygons for date: {polygons_filtered.size().getInfo()}")

    # 添加图层到地图
    
    Map.addLayer(images_filtered.median(), {"min": -25, "max": 0}, "Sentinel-1 Mosaic"+ date.format('YYYY-MM-dd').getInfo())
    Map.addLayer(polygons_filtered, {"color": "red"}, "Filtered Polygons" + date.format('YYYY-MM-dd').getInfo())

    # 返回图层信息
    return images_filtered, polygons_filtered




add the index filter for specific image, not always useful???? 

Yes

In [19]:
# 定义函数：对单个日期进行 Sentinel-1 图像筛选和采样
def process_date_index(date_ms, s1_index):
    """
    针对单个日期 (毫秒时间戳) 进行 Sentinel-1 筛选和采样。
    """
    try:
        # 转换为 ee.Date 格式并提取日期范围
        date = ee.Date(date_ms)
        date_str = date.format('YYYY-MM-dd').getInfo()  # 可读日期
        start_date = date.advance(-2, 'day')
        end_date = date.advance(2, 'day')
        
        # 打印日志信息
        print(f"Processing date: {date_str} (Range: {start_date.getInfo()} to {end_date.getInfo()})")

        # Sentinel-1 数据筛选
        s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                         .filterBounds(region)  # 替换为感兴趣区域
                         .filterDate(start_date, end_date)
                         #.filterDate(date,date)
                         .filter(ee.Filter.eq('instrumentMode', 'EW'))
                         .filterMetadata("system:index","contains", s1_index)
                         .filter(ee.Filter.Or(
                             ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'),
                             ee.Filter.listContains('transmitterReceiverPolarisation', 'HV')))
                       
        )
        
        # 检查是否有可用影像
        if s1_collection.size().getInfo() == 0:
            print(f"No valid Sentinel-1 images found for {date_str}.")
            return None
        print(f"Number of images found: {s1_collection.size().getInfo()}")
        # 归一化和波段处理
        processed_s1 = s1_collection.map(process_s1_image)

        # 按日期范围筛选多边形
        matching_polygons = sample_polygon.filter(ee.Filter.eq('time', date_ms)) 

        if matching_polygons.size().getInfo() == 0:
            print(f"No polygons found for date {date_str}.")
            return None
        
        print(f"Number of matching polygons: {matching_polygons.size().getInfo()}")
        
        # 可视化归一化波段
        vis_params = get_vis_params(processed_s1.median(), ['HH_norm', 'HV_norm', 'div_HH_HV_norm'])
        Map.addLayer(processed_s1, vis_params, f'S1 Normalized {date_str}', 0)
        
        visualize_issue(date_ms, sample_polygon, processed_s1)
        
        # 采样
        nbands_S1 = ['HH', 'HV', 'HH_norm', 'HV_norm', 'div_HH_HV_norm', 'diff_HH_HV_norm']
        
                  
        sampled = processed_s1.median().select(nbands_S1).sampleRegions(
            collection=matching_polygons,
            properties=['class'],  # 替换为实际属性名称
            scale=40,  # 根据需要调整
            geometries=True
        )
        print(f"Sampled {sampled.size().getInfo()} regions for date {date_str}.")
        return sampled

    except Exception as e:
        print(f"Error processing date: {e}")
        return None



In [28]:
# 定义函数：对单个日期进行 Sentinel-1 图像筛选和采样
def process_date(date_ms):
    """
    针对单个日期 (毫秒时间戳) 进行 Sentinel-1 筛选和采样。
    """
    try:
        # 转换为 ee.Date 格式并提取日期范围
        date = ee.Date(date_ms)
        date_str = date.format('YYYY-MM-dd').getInfo()  # 可读日期
        start_date = date.advance(-2, 'day')
        end_date = date.advance(2, 'day')
        
        # 打印日志信息
        print(f"Processing date: {date_str} (Range: {start_date.getInfo()} to {end_date.getInfo()})")

        # Sentinel-1 数据筛选
        s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                         .filterBounds(region)  # 替换为感兴趣区域
                         .filterDate(start_date, end_date)
                         #.filterDate(date,date)
                         .filter(ee.Filter.eq('instrumentMode', 'EW'))
                         .filter(ee.Filter.Or(
                             ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'),
                             ee.Filter.listContains('transmitterReceiverPolarisation', 'HV'))
                         )
        )
        
        # 检查是否有可用影像
        if s1_collection.size().getInfo() == 0:
            print(f"No valid Sentinel-1 images found for {date_str}.")
            return None
        print(f"Number of images found: {s1_collection.size().getInfo()}")
        # 归一化和波段处理
        processed_s1 = s1_collection.map(process_s1_image)

        # 按日期范围筛选多边形
        matching_polygons = sample_polygon.filter(ee.Filter.eq('time', date_ms)) 

        if matching_polygons.size().getInfo() == 0:
            print(f"No polygons found for date {date_str}.")
            return None
        
        print(f"Number of matching polygons: {matching_polygons.size().getInfo()}")
        
        # 可视化归一化波段
        vis_params = get_vis_params(processed_s1.median(), ['HH_norm', 'HV_norm', 'div_HH_HV_norm'])
        Map.addLayer(processed_s1, vis_params, f'S1 Normalized {date_str}', 0)
        
        visualize_issue(date_ms, sample_polygon, processed_s1)
        
        # 采样
        nbands_S1 = ['HH', 'HV', 'div_HH_HV_norm', 'diff_HH_HV_norm']
        sampled = processed_s1.median().select(nbands_S1).sampleRegions(
            collection=matching_polygons,
            properties=['class1'],  # 替换为实际属性名称
            scale=40,  # 根据需要调整
            geometries=True
        )
        print(f"Sampled {sampled.size().getInfo()} regions for date {date_str}.")
        return sampled

    except Exception as e:
        print(f"Error processing date: {e}")
        return None

## test

In [12]:
unique_dates = sample_polygon.aggregate_array('time').distinct().getInfo()  # 'time' 为 Unix 时间戳
formatted_dates = [ee.Date(date).format('YYYY-MM-dd').getInfo() for date in unique_dates]
print(f"Unique dates: {formatted_dates}")

Unique dates: ['2018-09-03', '2018-09-10', '2018-09-11', '2018-09-14', '2018-09-15', '2018-09-24', '2018-09-29', '2018-10-04', '2018-10-09', '2018-10-22', '2018-10-29', '2022-07-18']


### add the S1_list filter

In [17]:
S1_list = sample_polygon.aggregate_array('source').distinct().getInfo()
print(f"S1_list: {S1_list}")

S1_list: ['20180911T184432_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180914T074628_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180915T082725_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180924T080254_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180929T081103_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20181004T180321_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20181009T181232_S1B_AMSR2_IcechartGreenlandNorthAndCentralEast_Polygon', '20181022T180529_S1A_AMSR2_IcechartGreenlandNorthAndCentralEast_Polygon', '20181023T121709_S1A_AMSR2_IcechartGreenlandNorthWest_Polygon', '20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon', '20180910T081814_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon', '20180910T081914_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon', '20181023T121814_S1A_AMSR2_IcechartGreenlandNorthWest_Polygon', '20220718']


### 测试为什么lead采集不上

应该是ee.Filter or并集的设置存在问题

In [32]:
# 提取所有唯一日期
unique_dates = sample_polygon.aggregate_array('time').distinct().getInfo()  # 'time' 为 Unix 时间戳
formatted_dates = [ee.Date(date).format('YYYY-MM-dd').getInfo() for date in unique_dates]
date_ms = unique_dates[1]
s1_index = S1_list[1]
print(f"Unique dates: {unique_dates}")
print(f"s1_index: {s1_index}")
date = ee.Date(date_ms)
date_str = date.format('YYYY-MM-dd').getInfo()  # 可读日期
start_date = date.advance(-2, 'day')
end_date = date.advance(2, 'day')
    
print(f"Processing date: {date_str}")

# Sentinel-1 数据筛选
s1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                    .filterBounds(region)  # 替换为感兴趣区域
                    .filterDate(start_date, end_date)
                    #.filterDate(date,date)
                    .filter(ee.Filter.eq('instrumentMode', 'EW'))
                    .filterMetadata("system:index","contains", s1_index)
                    .filter(ee.Filter.Or(
                        ee.Filter.listContains('transmitterReceiverPolarisation', 'HH'),
                        ee.Filter.listContains('transmitterReceiverPolarisation', 'HV')))
                    
)

# 检查是否有可用影像
if s1_collection.size().getInfo() == 0:
    print(f"No valid Sentinel-1 images found for {date_str}.")

print(f"Number of images found: {s1_collection.size().getInfo()}")
# 归一化和波段处理
process_s1 = s1_collection.map(lambda image: image.clip(region)).map(process_s1_image)

# 按日期范围筛选多边形
matching_polygons = sample_polygon.filter(ee.Filter.eq('time', date_ms)) 

if matching_polygons.size().getInfo() == 0:
    print(f"No polygons found for date {date_str}.")
else:
    print(f"Number of matching polygons: {matching_polygons.size().getInfo()}")

# 可视化归一化波段
vis_params = get_vis_params(process_s1.median(), ['HH_norm', 'HV_norm', 'div_HH_HV_norm'])
Map.addLayer(process_s1, vis_params, f'S1 Normalized {date_str}', 0)
Map.centerObject(matching_polygons)
Map.addLayer(matching_polygons, {"color": "red"}, "Filtered Polygons" + date_str)


#visualize_issue(date, sample_polygon, process_s1)

point_vis_params = {
    "color": "yellow",  # 点样本颜色
    "pointSize": 5,     # 点的大小
    "opacity": 1        # 不透明度
}

# 在地图上添加 Sentinel-1 图像和多边形
Map.addLayer(process_s1, vis_params, f'S1 Normalized {date_str}', False)  # False 表示默认不显示图层
Map.centerObject(matching_polygons)  # 地图视角居中到多边形区域
Map.addLayer(matching_polygons, {"color": "red"}, "Filtered Polygons " + date_str)

# 突出显示点样本
if matching_polygons.size().getInfo() > 0:  # 确保点样本存在
    Map.addLayer(matching_polygons, point_vis_params, "Sampled Points " + date_str)
else:
    print("No points to display for this date.")
Map 

Unique dates: [1536649200000, 1536908400000, 1536994800000, 1537772400000, 1538204400000, 1538636400000, 1539068400000, 1540191600000, 1540796400000, 1535958000000, 1536562800000, 1657263600000]
s1_index: 20180914T074628
Processing date: 2018-09-14
Number of images found: 4
Number of matching polygons: 1


Map(bottom=676.0, center=[-64.16810689799152, 105.46875000000001], controls=(WidgetControl(options=['position'…

In [13]:
 # 采样
nbands_S1 = ['HH', 'HV', 'div_HH_HV_norm', 'diff_HH_HV_norm']
sampled = process_s1.median().select(nbands_S1).sampleRegions(
    collection=matching_polygons,
    properties=['class1'],  # 替换为实际属性名称
    scale=40,  # 根据需要调整
    geometries=True
)
print(f"Sampled {sampled.size().getInfo()} regions for date {date_str}.")
Map.addLayer(process_s1.median(), vis_params, f'S1 Normalized {date_str}', False)  # False 表示默认不显示图层
Map.addLayer(sampled, {
    "color": "red",  # 点样本颜色
    "pointSize": 5,     # 点的大小
    "opacity": 1        # 不透明度
}, "Sampled Points")


Sampled 542 regions for date 2022-07-18.


In [14]:
Map

Map(center=[71.27388998513885, -66.34214205643192], controls=(WidgetControl(options=['position', 'transparent_…

## sampling all points

In [13]:
unique_dates = sample_polygon.aggregate_array('time').distinct().getInfo()  # 'time' 为 Unix 时间戳
formatted_dates = [ee.Date(date).format('YYYY-MM-dd').getInfo() for date in unique_dates]
time1 = formatted_dates[1]
time1

'2018-09-10'

In [11]:
date_ms = unique_dates[1]
time1 = ee.Date(date_ms).format('YYYY-MM-dd')
print(time1.getInfo())
time1 = time1.getInfo().replace('-','')
time1

2018-09-14


'20180914'

In [12]:
S1_list = sample_polygon.aggregate_array('source').distinct().getInfo()
print(S1_list)
S1_list = [str(i)[:15] for i in S1_list]
res = [x for x in S1_list if time1 in x]
print(res)

['20180911T184432_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180914T074628_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180915T082725_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180924T080254_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20180929T081103_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20181004T180321_S1B_AMSR2_IcechartGreenlandCentralEast_Polygon', '20181009T181232_S1B_AMSR2_IcechartGreenlandNorthAndCentralEast_Polygon', '20181022T180529_S1A_AMSR2_IcechartGreenlandNorthAndCentralEast_Polygon', '20181023T121709_S1A_AMSR2_IcechartGreenlandNorthWest_Polygon', '20180903T082725_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon', '20180910T081814_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon', '20180910T081914_S1B_AMSR2_IcechartGreenlandNorthEast_Polygon', '20181023T121814_S1A_AMSR2_IcechartGreenlandNorthWest_Polygon', '20220718']
['20180914T074628']


In [20]:
# 定义主程序：对所有日期进行处理
def process_all_dates():
    """
    遍历所有日期进行 Sentinel-1 图像筛选和采样。
    """
    # 提取所有唯一日期
    unique_dates = sample_polygon.aggregate_array('time').distinct().getInfo()  # 'time' 为 Unix 时间戳
    formatted_dates = [ee.Date(date).format('YYYY-MM-dd').getInfo() for date in unique_dates]
    S1_list = sample_polygon.aggregate_array('source').distinct().getInfo()
    
    S1_list = [str(i)[:15] for i in S1_list] 
    
    #print(f"S1_list: {S1_list}")
    # 对每个日期运行采样逻辑
    #########################################################################
    #这里需要增加一个s1_index选项作为筛选条件
    #目前浪费双重循环，对一个日期可能存在两个index 
    #########################################################################
    all_samples = []
    for date_ms in unique_dates:
        time1 = ee.Date(date_ms).format('YYYY-MM-dd').getInfo().replace('-','')
        s1_indices = [x for x in S1_list if time1 in x]
        for s1_index in s1_indices:
            sampled = process_date_index(date_ms, s1_index)
            if sampled is not None:
                all_samples.append(sampled)
    # all_samples = []
    # for date_ms in unique_dates:
    #     sampled = process_date(date_ms)
    #     if sampled is not None:
    #         all_samples.append(sampled)
    #     sampled = process_date(date_ms)
    #     if sampled is not None:
    #         all_samples.append(sampled)

    # 合并所有采样结果
    if all_samples:
        combined_samples = ee.FeatureCollection(all_samples).flatten()
        print(f"Total sampled regions: {combined_samples.size().getInfo()}")
        print(f"Before flattening: {len(all_samples)}, After flattening: {combined_samples.size().getInfo()}")
        return combined_samples
    else:
        print("No valid samples found.")
        return ee.FeatureCollection([])


In [21]:
# 运行主程序
results = process_all_dates()
Map 

Processing date: 2018-09-03 (Range: {'type': 'Date', 'value': 1535785200000} to {'type': 'Date', 'value': 1536130800000})
Number of images found: 4
Number of matching polygons: 222
Visualizing for date: 2018-09-03
Number of images for date: 4
Number of polygons for date: 222
Sampled 222 regions for date 2018-09-03.
Processing date: 2018-09-10 (Range: {'type': 'Date', 'value': 1536390000000} to {'type': 'Date', 'value': 1536735600000})
Number of images found: 4
Number of matching polygons: 384
Visualizing for date: 2018-09-10
Number of images for date: 4
Number of polygons for date: 384
Sampled 152 regions for date 2018-09-10.
Processing date: 2018-09-10 (Range: {'type': 'Date', 'value': 1536390000000} to {'type': 'Date', 'value': 1536735600000})
Number of images found: 4
Number of matching polygons: 384
Visualizing for date: 2018-09-10
Number of images for date: 4
Number of polygons for date: 384
Sampled 384 regions for date 2018-09-10.
Processing date: 2018-09-11 (Range: {'type': 'Dat

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

## export sampled csv

In [22]:
combined_samples = results
# 检查是否有采样数据
if combined_samples.size().getInfo() > 0:
    task = ee.batch.Export.table.toAsset(
        collection=combined_samples,
        description='Sentinel1_Samples',
        assetId='projects/ee-my-yujiesun/assets/ee-my-SeaIce/SeaIce_Samples/Sentinel1_Sample_lead_add_Data_class_0226'
    )
    task.start()
    print("Export task started. Check your Asset for the file.")
else:
    print("No samples to export.")


Export task started. Check your Asset for the file.


In [23]:
combined_samples = results
# 检查是否有采样数据
if combined_samples.size().getInfo() > 0:
    task = ee.batch.Export.table.toDrive(
        collection=combined_samples,
        description='Sentinel1_Samples',
        folder='GEE_Exports',  # Google Drive 文件夹名称
        fileNamePrefix='Sentinel1_Sample_lead_add_Data_class_0226',
        fileFormat='CSV'
    )
    task.start()
    print("Export task started. Check your Google Drive for the file.")
else:
    print("No samples to export.")


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None))

# Not in GEEmap


## train

In [ ]:
# 为采样数据添加随机列，用于训练集和测试集拆分
#sampled_points = results.randomColumn('random', seed=42)
sampled_points = results
# 设置划分比例（例如：80% 训练集，20% 测试集）
split = 0.8
training = sampled_points.filter(ee.Filter.lt('random', split))
testing = sampled_points.filter(ee.Filter.gte('random', split))

# 输出训练集和测试集大小
print('Samples n =', sampled_points.size().getInfo())
print('Training n =', training.size().getInfo())
print('Testing n =', testing.size().getInfo())


NameError: name 'combined_samples' is not defined

In [110]:
# Generate and apply the RF
nbands_S1 = ['HH', 'HV', 'div_HH_HV_norm', 'diff_HH_HV_norm']

bands_class = nbands_S1
trainedClassifier = ee.Classifier.smileRandomForest(100, 2).train(
    features=training,
    classProperty='stage',
    inputProperties=bands_class
)

print(trainedClassifier.explain().getInfo(), 'importance')


EEException: Property 'class' of feature '0_00000000000000000003_0' is missing.